In [ ]:
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycountry
import seaborn as sns
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from geopy.geocoders import Photon
from IPython import get_ipython
from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
from iso3166 import countries_by_numeric

InteractiveShell.ast_node_interactivity = "all"

### READ DATASETS ###

df_soep = pd.read_parquet("soep2.gzip")
df_ivs = pd.read_stata("ivs_edited.dta", convert_categoricals=False)

In [ ]:
### IVS ###

# Country names are received from an API and saved as cache

CACHE_FILENAME = "country_data_cache.json"

if os.path.exists(CACHE_FILENAME):
    with open(CACHE_FILENAME, "r") as file:
        cache = json.load(file)
else:
    cache = {"numeric_to_alpha3": {}, "country_details": {}}

def numeric_to_alpha3(numeric_codes):
    str_codes = numeric_codes.astype(str)
    return str_codes.map(cache["numeric_to_alpha3"])

df_ivs["alpha3_codenum"] = numeric_to_alpha3(df_ivs["c_codenum"])

def get_country_details(alpha3_codes):
    def extract_detail(detail, x):
        try:
            return x.get(detail) if x is not None and isinstance(x, dict) else None
        except AttributeError:
            return None
    
    country_details = alpha3_codes.map(cache["country_details"])
    country_names = country_details.apply(lambda x: extract_detail("country_name", x))
    country_abbrs = country_details.apply(lambda x: extract_detail("country_abbr", x))

    return country_names, country_abbrs

df_ivs["country_name"], df_ivs["country_abbr"] = get_country_details(df_ivs["alpha3_codenum"])


# Save the cache back to the file
with open(CACHE_FILENAME, "w") as file:
    json.dump(cache, file)


In [ ]:
# Drop original column"
df_ivs.drop(columns=["c_codenum"])

# Create gender dummy variable"
df_ivs.sex.replace({1: "0", 2: "1"}, inplace=True)

# Create cohort variable"

bins = [18, 30, 45, 60, np.inf]
names = ["18-29", "30-44", "45-59", "60+"]

df_ivs["age_cohort"] = pd.cut(df_ivs["age"], bins, labels=names)

In [ ]:
# Weighting function using the IVS weights
def weighted_mean(df, val, weight):
    if df.empty or df[weight].isnull().all():
        return np.nan
    return (df[val] * df[weight]).sum() / df[weight].sum()

# Determine all unique values for countries, age cohorts, and years
unique_countries = df_ivs['alpha3_codenum'].unique()
unique_ages = df_ivs['age_cohort'].unique()
unique_years = range(df_ivs['year_survey'].min(), df_ivs['year_survey'].max() + 1)

# Create a DataFrame with all possible combinations of these values
all_combinations = pd.DataFrame([(country, age, year) for country in unique_countries for age in unique_ages for year in unique_years], 
                                columns=['alpha3_codenum', 'age_cohort', 'year_survey'])

# Merge with the original df_ivs dataset
merged_df = all_combinations.merge(df_ivs, on=['alpha3_codenum', 'age_cohort', 'year_survey'], how='left')

# Calculate the weighted mean for life satisfaction
def process_group(group):
    wm = weighted_mean(group, 'lifesatisfaction', 'weight')
    group['lifesatisfaction'] = wm
    return group

# Merge life weighted mean life satisfaction data
df_processed = merged_df.groupby(['alpha3_codenum', 'age_cohort', 'year_survey']).apply(process_group).reset_index(drop=True)

# Save country-year-cohort life satisfaction data in JSON file
json_data = {}

for _, row in df_processed.iterrows():
    country_code = row['alpha3_codenum']
    cohort = row['age_cohort']
    year = row['year_survey']
    lifesatisfaction = row['lifesatisfaction']
    
    if country_code not in json_data:
        json_data[country_code] = {}
    if cohort not in json_data[country_code]:
        json_data[country_code][cohort] = {}
    json_data[country_code][cohort][year] = lifesatisfaction

with open("lifesatisfaction_data.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)


In [ ]:
### SOEP ###

# Keep only rows with valid life satisfaction data
df_soep = df_soep[df_soep['satisfaction'].ge(0)]

# Calculate age(²) and map cohorts
df_soep["age"] = df_soep['syear'] - df_soep['gebjahr']

df_soep['age_sq'] = (df_soep['age'] ** 2)

df_soep["cohort"] = pd.cut(df_soep["age"], bins=[18, 30, 45, 60, np.inf], labels=["18-29", "30-44", "45-59", "60+"])

dummies = pd.get_dummies(df_soep['cohort'], prefix='cohort')

# Rename dummy columns for later use in Stata
new_dummy_columns = {col: col.replace('-', '_').replace('+', '') for col in dummies.columns}
dummies.rename(columns=new_dummy_columns, inplace=True)

# Add dummy series to SOEP dataframe
df_soep = pd.concat([df_soep, dummies], axis=1)

In [ ]:
# Transform weird Stata country labels
df_soep["corigin_str"] = df_soep["corigin_str"].astype(str).apply(lambda x: " ".join(x.split(" ")[1:]))

# Select only valid satisfaction values, only valid countries
df_soep = df_soep[df_soep['satisfaction'].ge(0) & ~df_soep["corigin_str"].isin(["No Answer", "Kosovo-Albania"])]

# Remove invalid countries (redundant?)
exclude_values = ["No Answer", "Kosovo-Albania"]
df_soep = df_soep[~df_soep["corigin_str"].isin(exclude_values)]

# Logarithm applied to income variable
df_soep['gen_income'] = df_soep['gen_income'].replace(-2, 1)
df_soep['gen_income'] = df_soep['gen_income'].replace(0, 1)
df_soep['gen_income'] = df_soep['gen_income'].apply(lambda x: x if x > 0 else None)

df_soep['gen_income'] = np.log(df_soep['gen_income'])

# Look up ISO country codes and save them in JSON for future use

def convert_to_alpha3(country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except AttributeError:
        try:
            return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
        except LookupError:
            return np.nan


if os.path.exists("country_mapping.json"):
    with open("country_mapping.json", "r") as file:
        country_to_alpha3 = json.load(file)
else:
    country_to_alpha3 = {}

unique_countries = df_soep["corigin_str"].unique()
for country in unique_countries:
    if country not in country_to_alpha3:
        country_to_alpha3[country] = convert_to_alpha3(country)

with open("country_mapping.json", "w") as file:
    json.dump(country_to_alpha3, file)

def map_origin_str_to_iso(row, mapping_dict):
    if row["corigin_str"] in mapping_dict:
        return mapping_dict[row["corigin_str"]]
    else:
        return row["corigin_iso"]

# Manually map unrecognized country codes

mapping_dict = {
    "Ex-Yugoslavia": "YUG",
    "Bosnia-Herzegovina": "BIH",
    "Eastern Europe": "?EE",
    "Columbia": "COL",
    "Moldavia": "MDA",
    "Kosovo": "XXK",
    "UAE": "ARE",
    "Benelux": "?BL",
    "Ivory Coast": "CIV",
    "No Nationality": "?NN",
    "Trinidad-Tobago": "TTO",
}

df_soep["corigin_iso"] = df_soep["corigin_str"].map(country_to_alpha3)
df_soep["corigin_iso"] = df_soep.apply(lambda row: mapping_dict.get(row["corigin_str"], row["corigin_iso"]), axis=1)


# Manually map wrong country names
df_soep["corigin_str"] = df_soep.corigin_str.replace("Columbia", "Colombia");
df_soep["corigin_str"] = df_soep.corigin_str.replace("Moldavia", "Moldova");


# Calculate years in Germany"

df_soep["y_in_germany"] = np.where(df_soep["immiyear"] >= 1, df_soep["syear"] - df_soep["immiyear"], np.nan)

In [ ]:
# Load data
oecd = pd.read_csv("thesis_oecd.csv")

# Adjust cohort mapping
age_cohort_mapping = {
    "18-29": "Young",
    "30-44": "Middle-aged",
    "45-59": "Middle-aged",
    "60+": "Old"
}

oecd['Age'] = oecd['Age'].map(age_cohort_mapping)

# Rename columns to the ones previously used
oecd = oecd.rename(columns={
    'LOCATION': 'alpha3_codenum',
    'Age': 'age_cohort',
    'Time': 'year_survey',
    'Value': 'lifesatisfaction'
})

# Save in JSON
json_data = {}
for _, row in oecd.iterrows():
    country_code = row['alpha3_codenum']
    cohort = row['age_cohort']
    year = row['year_survey']
    lifesatisfaction = row['lifesatisfaction']

    if country_code not in json_data:
        json_data[country_code] = {}
    if cohort not in json_data[country_code]:
        json_data[country_code][cohort] = {}
    json_data[country_code][cohort][year] = lifesatisfaction

with open("lifesatisfaction_data_oecd.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)


In [ ]:
# Load data
with open("lifesatisfaction_data.json", "r") as infile:
    lifesatisfaction_data = json.load(infile)
with open("lifesatisfaction_data_oecd.json", "r") as infile:
    oecd_data = json.load(infile)

# Merging the data
for country, cohorts in oecd_data.items():
    if country not in lifesatisfaction_data:
        lifesatisfaction_data[country] = {}
        
    for cohort, years in cohorts.items():
        if cohort not in lifesatisfaction_data[country]:
            lifesatisfaction_data[country][cohort] = {}
            
        for year, value in years.items():
            lifesatisfaction_data[country][cohort][year] = value

# Save merged data
with open("merged_lifesatisfaction_data.json", "w") as outfile:
    json.dump(lifesatisfaction_data, outfile, indent=4)


In [ ]:
# Load life satisfaction data
with open("merged_lifesatisfaction_data.json", "r") as file:
    df = json.load(file)

# Convert JSON to DataFrame
df_list = []
for country, cohorts in df.items():
    for cohort, years in cohorts.items():
        for year, value in years.items():
            df_list.append({'_key': country, 'Cohort/Year': f"{cohort}/{year}", 'Lifesatisfaction': value})
df = pd.DataFrame(df_list)

# Get all unique cohorts (excluding 'NaN')
all_cohorts = ['18-29', '30-44', '45-59', '60+']

# Identify countries with 'NaN' cohorts and replicate data for all cohorts
nan_cohort_countries = df[df['Cohort/Year'].str.startswith('NaN/')]['_key'].unique()
replicated_rows = []
for country in nan_cohort_countries:
    country_data = df[(df['_key'] == country) & (df['Cohort/Year'].str.startswith('NaN/'))].copy()
    for cohort in all_cohorts:
        new_data = country_data.copy()
        new_data['Cohort/Year'] = new_data['Cohort/Year'].str.replace('NaN', cohort)
        replicated_rows.append(new_data)

df = pd.concat([df] + replicated_rows, ignore_index=True)

# Remove rows with 'NaN' cohorts
df = df[~df['Cohort/Year'].str.startswith('NaN/')]

# Pivot the DataFrame
df_pivot = df.pivot_table(index='_key', columns='Cohort/Year', values='Lifesatisfaction', aggfunc='first')

# Apply Linear interpolation combined with LOCF/NOCB
df_interpolated = df_pivot.interpolate(method='linear', axis=1, limit_direction='both')

missing_oecd_countries = [country for country in oecd_countries if country not in df_interpolated.index]
if missing_oecd_countries:
    print("These OECD countries are missing from the df_interpolated dataframe:", missing_oecd_countries)

# Apply extrapolation only for OECD countries
for country in oecd_countries:
    df_interpolated.loc[country] = df_interpolated.loc[country].bfill().ffill()

df_melted = pd.melt(df_interpolated.reset_index(), id_vars=['_key'], value_name='Lifesatisfaction')

# Split Cohort and Year from the melted dataframe
df_melted[['Cohort', 'Year']] = df_melted['Cohort/Year'].str.split('/', n=1, expand=True)

filled_data = {}
for _, row in df_melted.iterrows():
    country = row['_key']
    cohort = row['Cohort']
    year = row['Year']
    value = row['Lifesatisfaction']

    if country not in filled_data:
        filled_data[country] = {}
    if cohort not in filled_data[country]:
        filled_data[country][cohort] = {}
    filled_data[country][cohort][year] = value

# Save the filled data as a JSON
with open("lifesatisfaction_data_filled.json", "w") as outfile:
    json.dump(filled_data, outfile, indent=4)


In [ ]:
# Rename columns to prepare merge
df_ivs["alpha3_codenum"] = df_ivs["alpha3_codenum"].astype(str)
df_ivs = df_ivs.reset_index()  # reset index if your columns are currently indices
df_ivs = df_ivs.rename(
    columns={
        "alpha3_codenum": "corigin_iso",
        "year_survey": "syear",
        "age_cohort": "cohort",
        "lifesatisfaction": "satisf_peers",
    }
)
df_ivs.set_index(["corigin_iso", "cohort", "syear"], inplace=True)

# Drop missing data"

df_ivs.dropna(subset=["index", "satisf_peers"], inplace=True)
df_ivs.reset_index(inplace=True)
df_ivs = df_ivs[["corigin_iso", "syear", "cohort", "sex", "satisf_peers", "age", "subj_health", "educ_recoded", "educ_isced97", "educ_isced11", "educ_casmin", "occ_status_siops", "occ_status_isei", "occ_status_epg11", "empl_status"]]


In [ ]:
# Remove dumplicate that sometimes appear when running the same code cell twice
def resolve_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols

# Applying the function to resolve duplicate column names
resolve_duplicate_columns(df_soep)

# Save temporary datasets (to prevent having to run the full code each time)
df_soep.to_parquet("soep_unmerged.gzip", compression='gzip')
df_ivs.to_parquet("ivs_unmerged.gzip")

In [ ]:
# Load temporary datasets
df_soep = pd.read_parquet("soep_unmerged.gzip")
df_ivs = pd.read_parquet("ivs_unmerged.gzip")


# Merge IVS and OECD data from JSON to SOEP

with open("lifesatisfaction_data_filled.json", "r") as file:
    json_data = json.load(file)

df_list = []
for country, cohorts in json_data.items():
    for cohort, years in cohorts.items():
        for year, value in years.items():
            df_list.append({
                'corigin_iso': country,
                'cohort': cohort,
                'syear': int(year),
                'satisf_peers': value
            })

df_json = pd.DataFrame(df_list)

df_soep = df_soep.merge(df_json, on=['corigin_iso', 'syear', 'cohort'], how='left')

# Create satisfaction to peers difference variable
df_soep["satisf_diff"] = df_soep["satisfaction"] - df_soep["satisf_peers"]

In [ ]:
# Create religion dummies
df_soep['rel_christ'] = np.where(df_soep['religion'].isin([1, 2, 3, 7]), 1, 0)
df_soep['rel_musl'] = np.where(df_soep['religion'].isin([4, 8, 9, 10]), 1, 0)
df_soep['rel_unaffiliated'] = np.where(df_soep['religion'] == 6, 1, 0)
df_soep['rel_other'] = np.where(df_soep['religion'] == 5, 1, 0)
df_soep['rel_unknown'] = np.where(~df_soep['religion'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), 1, 0)



In [ ]:
geolocator = Photon(user_agent="myGeocoder")

# Load existing coordinate cache, if available
try:
    with open("country_coords.json", "r") as f:
        coordinate_cache = json.load(f)
except FileNotFoundError:
    coordinate_cache = {}

def get_coordinates(country):
    location = geolocator.geocode(country)
    if location:
        return location.latitude, location.longitude
    return None, None  # Return tuple of None values for missing coordinates

# Get coordinates for all countries
unique_countries = df_soep["corigin_str"].unique()
coords_dict = {country: get_coordinates(country) for country in unique_countries}

# Update cache
coordinate_cache.update(coords_dict)
with open("country_coords.json", "w") as f:
    json.dump(coordinate_cache, f)

# Map coordinates to countries
df_soep['country_coords'] = df_soep['corigin_str'].map(coordinate_cache)

# Ensure that every value in df_soep['country_coords'] is a tuple
df_soep['country_coords'] = df_soep['country_coords'].apply(
    lambda x: x if isinstance(x, tuple) else (None, None)
)

# Calculate distance to Germany
germany_coords = coordinate_cache.get("Germany", (None, None))
if germany_coords == (None, None):
    germany_coords = get_coordinates("Germany")
    coordinate_cache["Germany"] = germany_coords
    with open("country_coords.json", "w") as f:
        json.dump(coordinate_cache, f)

# Calculate the distance using the updated column
df_soep['dist_origin'] = df_soep['country_coords'].apply(
    lambda x: geodesic(germany_coords, x).kilometers if x[0] is not None and x[1] is not None else None
)

# Drop the temporary column
df_soep.drop('country_coords', axis=1, inplace=True)


In [ ]:
# Transforming variables

df_soep['bad_health'] = df_soep['curr_health'].apply(lambda x: 1 if x in [4, 5] else (0 if x in [1, 2, 3] else np.nan))

df_soep['lang_profic'] = df_soep.apply(lambda row: 
                                      1 if row['lang_oral'] in [1, 2] or 
                                         row['lang_usl'] in [1, 3] or 
                                         row['ger_newspaper'] in [3, 4, 5] or 
                                         row['lang_fam'] in [1, 4] or 
                                         row['lang_friends'] == 1 or 
                                         row['ger_premig'] in [1, 2] or 
                                         row['lang_oral2'] in [1, 2] 
                                      else (0 if row['lang_oral'] in [3, 4, 5] else np.nan), 
                                      axis=1)
df_soep['lang_profic'] = df_soep.groupby('pid')['lang_profic'].ffill()

df_soep['edu_collg'] = df_soep['edu_collg'].apply(lambda x: 1 if x > 0 else (0 if x == 0 else np.nan))

df_soep['edu_novocat'] = df_soep['edu_novocat'].apply(lambda x: 1 if x == 1 else (0 if x in [2, 3] else np.nan))

df_soep['gen_wpartner'] = df_soep['gen_family'].apply(lambda x: 1 if x in [1, 7] else (0 if x >= 0 else np.nan))

df_soep['gen_seppart'] = df_soep['gen_family'].apply(lambda x: 1 if x in [2, 6, 7] else (0 if x >= 0 else np.nan))

df_soep['gen_wid_div'] = np.where(df_soep['gen_family'].isin([4, 5]), 1, 
                                  np.where(df_soep['gen_family'] < 0, np.nan, 0))

df_soep['gen_single'] = df_soep['gen_family'].apply(lambda x: 1 if x == 3 else (0 if x >= 0 else np.nan))

df_soep['gen_notrain'] = df_soep['gen_edu97'].apply(lambda x: 1 if x in [0, 1, 2] else (0 if x >= 0 else np.nan))

df_soep['gen_tertiary'] = df_soep['gen_edu97'].apply(lambda x: 1 if x in [5, 6] else (0 if x >= 0 else np.nan))

df_soep['gen_employed'] = df_soep['gen_emplmnt'].apply(lambda x: 1 if x in [1, 2] else (0 if x >= 3 else np.nan))

df_soep['gen_vocation'] = df_soep['gen_emplmnt'].apply(lambda x: 1 if x == 3 else 0)

df_soep['gen_retired'] = df_soep['gen_typeunempl'].apply(lambda x: 1 if x == 2 else (0 if x > 2 else np.nan))
df_soep['gen_retired'] = df_soep.groupby('pid')['gen_retired'].ffill()

df_soep['gen_regunempl'] = df_soep['gen_typeunempl'].apply(lambda x: 1 if x == 6 else (0 if x != 6 and x > 0 else np.nan))

df_soep['refugee'] = df_soep['ig_grp'].apply(lambda x: 1 if x == 5 else (np.nan if x == -5 else 0))

df_soep['ig_cont_fam'] = df_soep['ig_cont_fam'].apply(lambda x: 1 if x == 1 else (0 if x == 2 else np.nan))

df_soep['feel_german'] = df_soep['feel_german'].apply(lambda x: 1 if x in [1, 2] else (0 if x in [3, 4, 5] else np.nan))

df_soep['visit_germ'] = df_soep['visit_germ'].apply(lambda x: 1 if x == 1 else (0 if x == 2 else np.nan))

df_soep.loc[df_soep['edu_years'] < 0, 'edu_years'] = np.nan

df_soep = df_soep.sort_values(by=['pid', 'syear'])

# Forwards fill variables
df_soep['feel_german'] = df_soep.groupby('pid')['feel_german'].ffill()
df_soep['visit_germ'] = df_soep.groupby('pid')['visit_germ'].ffill()

df_soep['want_stay'] = df_soep.apply(lambda row: 1 if row['stay_ger'] == 1 or row['stay_ger13'] == 1 else (0 if row['stay_ger'] == 2 or row['stay_ger13'] == 2 else np.nan), axis=1)

df_soep['german'] = df_soep['gen_nation'].apply(lambda x: 1 if x == 1 else 0)

columns_to_check = [
    "sat_work", "sat_hhinc", "sat_dwell", 
    "sat_leisure", "sat_family", "sat_sleep"
]

for column in columns_to_check:
    df_soep.loc[~df_soep[column].between(0, 10), column] = np.nan

df_soep['discrimination'] = df_soep.apply(
    lambda row: 1 if any(x in [1, 2] for x in [row['disadv_origin1'], row['disadv_origin2']]) 
                else (0 if any(x in [3, -2, -1] for x in [row['disadv_origin1'], row['disadv_origin2']]) 
                      else np.nan), 
    axis=1
)

df_soep['discrimination'] = df_soep.groupby('pid')['discrimination'].ffill()

df_soep['dist_city'] = df_soep['dist_city'].apply(lambda x: 1 if x in [1, 2] else 0)
df_soep['dist_city'] = df_soep.groupby('hid')['dist_city'].ffill()

def assign_social(row):
    if (row['lonely'] in [3, 4] or 
        row['nbh_relation'] in [1, 2, 3] or 
        row['nbh_freq'] in [1, 2] or 
        row['cl_friends'] > 2):
        return 1
    elif (row['lonely'] in [1, 2] or 
          row['cl_friends'] in [0, 1]):
        return 0
    else:
        return np.nan

df_soep['social'] = df_soep.apply(assign_social, axis=1)

df_soep['ig_cont_fam'] = df_soep['ig_cont_fam'].apply(lambda x: 1 if x == 1 else (0 if x in [0, -2] else np.nan))

df_soep['xenophobia'] = df_soep['xenophobia'].apply(lambda x: 1 if x in [1, 2] else (0 if x in [3, -1, -2] else np.nan))

df_natives = df_soep[df_soep['migback'] == 1]

def weighted_satisfaction(group):
    return sum(group['satisfaction'] * group['hochrechnungsfaktor']) / sum(group['hochrechnungsfaktor'])

result = df_natives.groupby(['syear', 'cohort']).apply(weighted_satisfaction).reset_index(name='satisf_natives')

df_soep = pd.merge(df_soep, result, on=['syear', 'cohort'], how='left')

df_soep['satisf_ndiff'] = df_soep['satisfaction'] - df_soep['satisf_natives']

df_soep['yig_sq'] = df_soep['y_in_germany'] ** 2
df_soep['edu_sq'] = df_soep['edu_years'] ** 2

df_soep['num_children'] = df_soep['num_children'].apply(lambda x: 0 if x in [-1, -2] else (x if x >= 0 else np.nan))

columns_to_fill = ['social', 'feel_german', 'discrimination', 'xenophobia']

# Forwards fill with mode
df_soep[columns_to_fill] = df_soep[columns_to_fill].astype('float32')

def compute_mode(series):
    mode_series = series.mode()
    if len(mode_series) == 0:
        return np.nan 
    else:
        return mode_series.iloc[0]

for col in columns_to_fill:
    mode_by_pid = df_soep.groupby('pid')[col].transform(compute_mode)
    df_soep[col].fillna(mode_by_pid, inplace=True)

In [ ]:
# Right-wing activity in federal state, merge into SOEP, log participants number

rwe = pd.read_parquet("rwe.gzip")

mapping = {
    1: "Schleswig-Holstein",
    2: "Hamburg",
    3: "Niedersachsen",
    4: "Bremen",
    5: "Nordrhein-Westfalen",
    6: "Hessen",
    7: "Rheinland-Pfalz,Saarland",
    8: "Baden-Württemberg",
    9: "Bayern",
    10: "Saarland",
    11: "Berlin",
    12: "Brandenburg",
    13: "Mecklenburg-Vorpommern",
    14: "Sachsen",
    15: "Sachsen-Anhalt",
    16: "Thüringen",
    -1: "Unknown", 
}

df_soep["state"].fillna(-1, inplace=True)
df_soep["state"] = df_soep["state"].map(mapping)
rwe["state"] = rwe["state"].astype(str)
df_soep = pd.merge(df_soep, rwe, on=["syear", "state"], how="outer")
df_soep['logrwe'] = np.where(df_soep['total_participants'] > 0, np.log(df_soep['total_participants']), np.nan)
df_soep.loc[(df_soep['syear'] >= 2005) & (df_soep['syear'] <= 2020) & (df_soep['logrwe'].isna()), 'logrwe'] = 0


In [ ]:
# Variable transformation, select reasonable birth years

df_soep["sex"] = df_soep["sex"] - 1

df_soep.loc[df_soep['cl_friends'] < 0, 'cl_friends'] = np.nan
df_soep.loc[df_soep['lonely'] < 0, 'lonely'] = np.nan
df_soep.loc[df_soep['sex'] < 0, 'sex'] = np.nan


df_soep = df_soep.loc[df_soep["gebjahr"].ge(1900)]

# East/west dummy
df_soep["eastwest"] = df_soep["eastwest"] - 1
df_soep.loc[df_soep["eastwest"] < 0, "eastwest"] = np.nan

# Calculate age at migration"
df_soep['migr_age'] = df_soep.apply(lambda row: row['immiyear'] - row['gebjahr'] if row['immiyear'] > 1900 else np.nan, axis=1)

In [ ]:
# Create continent dummies with help of alpha-3 codes

CACHE_FILE = "country_continent_cache.json"

# Load cache if exists
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        cache = json.load(f)
else:
    cache = {}

def alpha3_to_alpha2(alpha_3):
    """Converts ISO 3166-1 alpha-3 to ISO 3166-1 alpha-2."""
    return pycountry.countries.get(alpha_3=alpha_3).alpha_2

def country_to_continent(country_code):
    if country_code in cache:
        return cache[country_code]
    
    try:
        alpha_2 = alpha3_to_alpha2(country_code)
        continent_code = country_alpha2_to_continent_code(alpha_2)
        continent_name = convert_continent_code_to_continent_name(continent_code)
        cache[country_code] = continent_name
        return continent_name
    except Exception as e:
        return None

unique_countries_not_in_cache = df_soep.loc[~df_soep["corigin_iso"].isin(cache), "corigin_iso"].unique()
continent_mapping = {country: country_to_continent(country) for country in unique_countries_not_in_cache}

df_soep["continent"] = df_soep["corigin_iso"].map(lambda x: cache.get(x, continent_mapping.get(x)))

# Update the cache file
with open(CACHE_FILE, "w") as f:
    json.dump(cache, f)

# Add dummies to SOEP
continent_dummies = pd.get_dummies(df_soep["continent"], prefix="continent").astype(int)
df_soep = pd.concat([df_soep, continent_dummies], axis=1)

# Replace spaces for future use
df_soep.columns = df_soep.columns.str.replace(" ", "_")

continent_dict = {
    "?EE": "Europe",
    "YUG": "Europe", 
    "CRI": "North America",  
    "MNG": "Asia",  
    "MOZ": "Africa",  
    "BOL": "South America",  
    "?BL": "Europe",  
    "SLV": "North America",  
    "TCD": "Africa",  
    "?NN": "No Nationality",
    "MUS": "Africa",  
    "TTO": "North America",  
    "CYP": "Europe",  
    "NOR": "Europe",  
    "LBR": "Africa",  
    "DOM": "North America",  
    "SEN": "Africa",  
     None: "Unknown",
    "HND": "North America",  
    "SGP": "Asia",  
    "ECU": "South America",
    "UGA": "Africa",  
    "NZL": "Oceania",  
    "BWA": "Africa",  
    "MMR": "Asia",  
    "HTI": "North America",  
    "CIV": "Africa",  
    "URY": "South America",  
    "XXK": "Europe",
    "YEM": "Asia",  
    "ZMB": "Africa",  
    "KHM": "Asia",  
    "LSO": "Africa",  
    "MYS": "Asia",  
    "SUR": "South America",  
    "ZWE": "Africa",  
    "NIC": "North America",  
    "MLI": "Africa", 
    "RWA": "Africa",  
    "NPL": "Asia",  
    "WSM": "Oceania",  
    "DJI": "Africa",  
    "MDV": "Asia",  
    "MDG": "Africa", 
    "BFA": "Africa", 
    "HKG": "Asia",  
    "TZA": "Africa",  
    "GUY": "South America",  
    "NER": "Africa",  
    "BEN": "Africa",  
    "QAT": "Asia"   
}

# Apply manually mapped continents
df_soep['continent'] = df_soep['corigin_iso'].map(continent_dict).where(
    df_soep['continent'].isna(), 
    df_soep['continent']
)


In [ ]:
# Save datasets

# Identify OECD countries
oecd_countries = oecd['alpha3_codenum'].unique().tolist()

df_soep.to_parquet("df_soep2.gzip")
oecd_df = df_soep[df_soep['corigin_iso'].isin(oecd_countries)]
oecd_df.to_parquet("oecd.gzip")
oecd_df.to_csv("oecd.csv")